<a href="https://colab.research.google.com/github/Kropotikin/Introduction_to_Machine_Learning_with_Python/blob/main/Cap4_Represent_dados_feature_engeneering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[GitHub](https://github.com/Kropotikin/introduction_to_ml_with_python/blob/master/04-representing-data-feature-engineering.ipynb)

<hr color = 'B7FF80'>

#<h1 align = 'right'><font color = 'c77dff' size = +3.5><b>Preparando ambiente</b></font></h1>
<hr color = 'B7FF80'>

In [1]:
!pip install mglearn

     |████████████████████████████████| 540 kB 4.2 MB/s 
  Created wheel for mglearn: filename=mglearn-0.1.9-py2.py3-none-any.whl size=582639 sha256=a0ae7c6bedfa9f97ab2a93aa359847e554eb11d30249264622dcc00a10e8cc00
  Stored in directory: /root/.cache/pip/wheels/f1/17/e1/1720d6dcd70187b6b6c3750cb3508798f2b1d57c9d3214b08b
Successfully built mglearn


In [2]:
import sys
print(f"Python version: {sys.version}")
import pandas as pd
print(f"Pandas version: {pd.__version__}")
import numpy as np
print(f"Numpy version: {np.__version__}")
import matplotlib
print(f"Matplotlib version: {matplotlib.__version__}")
import matplotlib.pyplot as plt
import mglearn
print(f"Mglearn version: {mglearn.__version__}")
import scipy as sp
print(f"Scipy version: {sp.__version__}")
import sklearn
print(f"Sklearn version: {sklearn.__version__}")

Python version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Pandas version: 1.3.5
Numpy version: 1.21.6
Matplotlib version: 3.2.2
Mglearn version: 0.1.9
Scipy version: 1.4.1
Sklearn version: 1.0.2


In [3]:
# desativar avisos
import warnings
warnings.filterwarnings('ignore')

In [4]:
# habilitar a plotagem no notebook
%matplotlib inline
#from preamble import *
plt.rcParams['image.cmap'] = "gray"

In [5]:
# retira notação científica
pd.options.display.float_format = '{:.4f}'.format

<hr color = 'B7FF80'>

#<h1 align = 'right'><font color = 'c77dff' size = +3.5><b>Representação de dados e Engenharia de Recursos</b></font></h1>
<hr color = 'B7FF80'>

Até agora, assumimos que nossos dados vêm como uma matriz bidimensional de números de ponto flutuante, onde cada coluna é um recurso contínuo que descreve os pontos de dados. Para muitos aplicações, não é assim que os dados são coletados. Um tipo de recurso particularmente comum são os recursos categóricos. Também conhecidos como recursos discretos,
estes geralmente não são numéricos. A distinção entre características categóricas e características contínuas é análoga à distinção entre classificação e regressão, apenas no lado da entrada e não no lado da saída. Exemplos de recursos contínuos que vimos são brilhos de pixel e medidas de tamanho de flores de plantas. Exemplos de características categóricas são a marca de um produto, a cor de um produto ou o departamento (livros, roupas, hardware) em que é vendido. Todas essas são propriedades que podem descrever um produto, mas não variam em um maneira contínua. Um produto pertence ao departamento de roupas ou ao departamento de livros. Não há meio solo entre livros e roupas, e nenhuma ordem natural para as diferentes categorias (livros não é maior ou menor que roupas, hardware não é entre livros e roupas, etc.).

Independentemente dos tipos de recursos em que seus dados consistem, a forma como você os representa pode ter um efeito enorme no desempenho dos modelos de aprendizado de máquina. Vimos nos Capítulos 2 e 3 que o dimensionamento dos dados é importante. Em outras palavras, se você não redimensionar seus dados (digamos, para a variação da unidade), fará diferença se você representa uma medida em centímetros ou polegadas. Também vimos no Capítulo 2 que pode ser útil aumentar seus dados com recursos adicionais, como adicionar interações (produtos) de recursos ou polinômios mais gerais.

A questão de como representar melhor seus dados para um aplicativo específico é conhecida como *feature engineering* (engenharia de recursos) e é uma das principais tarefas dos cientistas de dados e praticantes de aprendizado de máquina que tentam resolver problemas do mundo real. Representar seus dados da maneira correta pode ter uma influência maior no desempenho de um modelo supervisionado do que os parâmetros exatos que você escolher.

Neste capítulo, examinaremos primeiro o caso importante e muito comum de recursos categóricos e, em seguida, daremos alguns exemplos de transformações úteis para combinações específicas de recursos e modelos.

#**Variáveis ​​categóricas**

Como exemplo, usaremos o conjunto de dados de renda de adultos nos Estados Unidos, derivado do banco de dados do censo de 1994. A tarefa do conjunto de dados adulto é prever se um trabalhador tem uma renda superior a US$ 50.000 ou inferior a US$ 50.000. As características neste conjunto de dados incluem a idade dos trabalhadores, como eles são empregados (autônomos, indústria privada
funcionário, funcionário do governo, etc.), sua educação, seu gênero, suas horas de trabalho por semana, ocupação e muito mais.

A tarefa é formulada como uma tarefa de classificação com as duas classes sendo renda <=50k e >50k. Também seria possível prever a renda exata e fazer disso uma tarefa de regressão. No entanto, isso seria muito mais difícil, e a divisão de 50K é interessante de entender por conta própria.

Neste conjunto de dados, a idade e as horas por semana são características contínuas, que sabemos como tratar. As características da classe de trabalho, educação, sexo e ocupação são categóricas, no entanto. Todos eles vêm de uma lista fixa de valores possíveis, em oposição a um intervalo, e denotam uma propriedade qualitativa, em oposição a uma quantidade.

Como ponto de partida, digamos que queremos aprender um classificador de regressão logística nesses dados. Sabemos do Capítulo 2 que uma regressão logística faz previsões, ŷ, usando a seguinte fórmula:
>ŷ = w[0] * x[0] + w[1] * x[1] + ... + w[p] * x[p] + b > 0

onde w[i] e b são os coeficientes aprendidos do conjunto de treinamento e x[i] são os recursos de entrada. Esta fórmula faz sentido quando x[i] são números, mas não quando x[2] é "Mestres" ou "Bacharelados". Claramente, precisamos representar nossos dados em alguns maneira ao aplicar a regressão logística. A próxima seção explicará como podemos superar este problema.

#**Hot-Encoding (Dummy Variables)**

De longe, a maneira mais comum de representar variáveis categóricas é usando a codificação one-hotencoding ou one-out-of-N, também conhecida como variáveis fictícias. A ideia por trás das variáveis fictícias é substituir uma variável categórica por um ou mais novos recursos que podem ter os valores 0 e 1. Os valores 0 e 1 fazem sentido na fórmula para classificação binária linear (e para todos os outros modelos no scikit-learn), e podemos representar qualquer número de categorias introduzindo um novo recurso por categoria, conforme descrito aqui.

Digamos que para o recurso de classe de trabalho temos valores possíveis de "Government Employee", "Private Employee", "Self Employed" e "Self Employed Incorporated". Para codificar esses quatro valores possíveis, criamos quatro novos recursos, chamados de "Government Employee", "Private Employee", "Self Employed", and "Self Employed Incorporated". Um recurso é 1 se a classe de trabalho para essa pessoa tiver o valor correspondente e 0 caso contrário, exatamente um dos quatro novos recursos será 1 para cada ponto de dados. É por isso que isso é chamado de codificação one-hot ou one-out-of-N.

Um único recurso é codificado usando quatro novos recursos. Ao usar esses dados em um algoritmo de aprendizado de máquina, descartamos o recurso de classe de trabalho original e mantemos apenas os recursos de 0 a 1.

>A codificação one-hot que usamos é bastante semelhante, mas não idêntica, à
a codificação fictícia usada nas estatísticas. Para simplificar, codificamos
cada categoria com um recurso binário diferente. Em estatística, é comum codificar uma característica categórica com k diferentes valores possíveis em k–1 características (a última é representada como todos os zeros). Isso é feito para simplificar a análise (mais tecnicamente, isso evitará tornar a matriz de dados deficiente em classificação).

Existem duas maneiras de converter seus dados em uma codificação one-hot de variáveis categóricas, usando pandas ou scikit-learn. No momento em que escrevo, usar pandas é um pouco mais fácil, então vamos por esse caminho. Primeiro carregamos os dados usando pandas de um arquivo de valores separados por vírgula (CSV):

In [6]:
import os
# O arquivo não tem cabeçalhos nomeando as colunas, então passamos header=None
# e forneçemos os nomes das colunas explicitamente em "names"
adult_path = os.path.join(mglearn.datasets.DATA_PATH, "adult.data")
data = pd.read_csv(
    adult_path, header = None, index_col = False,
    names=['age', 'workclass', 'fnlwgt', 'education',  'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'gender',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
           'income'])
# Para fins de ilustração, selecionamos apenas algumas das colunas
data = data[['age', 'workclass', 'education', 'gender', 'hours-per-week',
             'occupation', 'income']]
# IPython.display permite uma boa formatação de saída no notebook Jupyter
display(data.head())

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


#**Verificando dados categóricos codificados em string**

Depois de ler um conjunto de dados como esse, geralmente é bom verificar primeiro se uma coluna realmente contém dados categóricos significativos. Ao trabalhar com dados que foram inseridos por humanos (digamos, usuários em um site), pode não haver um conjunto fixo de categorias e diferenças na ortografia e capitalização podem exigir pré-processamento. Por exemplo, pode ser que algumas pessoas especifiquem o gênero como “masculino” e outras como “homem”, e talvez queiramos representar essas duas entradas usando a mesma categoria. Uma boa maneira de verificar o conteúdo de uma coluna é usar a função value_counts de uma série pandas (o tipo de uma única coluna em um DataFrame), para nos mostrar quais são os valores exclusivos e com que frequência eles aparecem:

In [8]:
data['gender'].value_counts()

 Male      21790
 Female    10771
Name: gender, dtype: int64

Podemos ver que existem exatamente dois valores para gênero neste conjunto de dados, Male e Female, o que significa que os dados já estão em um bom formato para serem representados usando a codificação one-hot. Em uma aplicação real, você deve observar todas as colunas e verificar seus valores. Vamos pular isso aqui por uma questão de brevidade.

Existe uma maneira muito simples de codificar os dados em pandas, usando a função `get_dummies`. A função `get_dummies` transforma automaticamente todas as colunas que possuem tipo de objeto (como strings) ou são categóricas (que é um conceito especial de pandas sobre o qual ainda não falamos):

In [9]:
print(f"Original features:\n{list(data.columns)}\n")
data_dummies = pd.get_dummies(data)
print(f"Features after get_dummies:\n {list(data_dummies.columns)}")

Original features:
['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income']

Features after get_dummies:
 ['age', 'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'gender_ Female', 'gender_ Male', 'occupation_ ?', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners', 'occupation_ Machine-op-ins

Você pode ver que a idade dos recursos contínuos e as horas por semana não foram alteradas, enquanto os recursos categóricos foram expandidos em um novo recurso para cada valor possível:

In [10]:
data_dummies.head()

,age,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,income_ <=50K,income_ >50K
0,39,40,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,50,13,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,38,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,53,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,28,40,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


Agora podemos usar o atributo values para converter o data_dummies DataFrame em um array NumPy e, em seguida, treinar um modelo de aprendizado de máquina nele. Tenha cuidado para separar a variável de destino (que agora está codificada em duas colunas de renda) dos dados antes de treinar um modelo. Incluindo a variável de saída, ou alguma propriedade derivada de a variável de saída, na representação do recurso é um erro muito comum na construção de modelos de aprendizado de máquina supervisionados.

>Tenha cuidado: a indexação de coluna em pandas inclui o final do intervalo, portanto, 'age':'occupation_ Transport-moving' inclui ocupação_Transport-moving. Isso é diferente de fatiar um array NumPy, onde o final de um intervalo não é incluído: por exemplo, np.arange(11)[0:10] não inclui a entrada com índice 10.

Nesse caso, extraímos apenas as colunas que contêm recursos, ou seja, todas as colunas de idade a ocupação_ Transporte-movimento. Este intervalo contém todos os recursos, mas não o destino:

In [15]:
features = data_dummies.loc[:, 'age': 'occupation_ Transport-moving']
# Extraia matrizes NumPy
X = features.values
y = data_dummies['income_ >50K'].values
print(f"X.shape: {X.shape}\ny.shape: {y.shape}")

X.shape: (32561, 44)
y.shape: (32561,)


Agora os dados são representados de uma maneira que o scikit-learn pode trabalhar e podemos proceder como de costume:

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print(f"Test score: {logreg.score(X_test, y_test):.2%}")

Test score: 80.68%


>Neste exemplo, chamamos get_dummies em um DataFrame contendo tanto os dados de treinamento quanto os de teste. Isso é importante para garantir que os valores categóricos sejam representados da mesma maneira no conjunto de treinamento
e o conjunto de teste.

>Imagine que temos os conjuntos de treinamento e teste em dois conjuntos de dados diferentes. Se o valor "Private Employee" para o recurso work class
não aparece no conjunto de teste, o pandas assumirá que existe apenas
três valores possíveis para este recurso e criará apenas três novos
características fictícias. Agora nossos conjuntos de treinamento e teste têm  vários recursos, e não podemos aplicar o modelo que aprendemos o conjunto de treinamento para o conjunto de teste. Pior ainda, imagine que o recurso work class tem os valores "Government Employee" e "Private Employee" no conjunto de treinamento, e "Self Employee" e "Self Employed Incorporated" no conjunto de teste. Em ambos os casos, pandas criará dois novos recursos fictícios, então os dados codificados terão o mesmo número de recursos. No entanto, as duas características fictícias têm significados totalmente diferentes no treinamento e conjuntos de teste. A coluna que significa "Government Employee" para o conjunto de treinamento codificaria "Self Employee" para o conjunto de teste. Se construíssemos um modelo de aprendizado de máquina com esses dados, funcionaria muito mal, porque assumiria que as colunas significam a mesma coisa (porque elas estão na mesma posição) quando na verdade elas significam coisas muito diferentes. Para corrigir isso, chame get_dummies em  DataFrame que contém os pontos de dados de treinamento e teste, ou certifique-se de que os nomes das colunas sejam os mesmos para o treinamento e conjuntos de teste depois de chamar get_dummies, para garantir que eles tenham a mesma semântica.